<a href="https://colab.research.google.com/github/Junseokee/Deep_Learning/blob/main/220916_BERT_Basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 숫자세계로 떠난 자연어

## 미리 학습된 언어 모델

최근 BERT, GPT 같은 모델이 주목을 받게된 이유는 성능,
이 모델들은 문서분류, 개체명 인식 등 어떤 태스크든 성능이 좋음
GPT는 순방향 언어모델이다. 앞단어를 통해 뒷단어 예측 (문장 생성에 강점)   
BERT는 마스크언어모델이다. 빈칸(마스크) 다음에 뭐가 올지 유추하며 학습하는 방법 (빈칸 채우기에 강점)   

스킵-그램모델
- 앞뒤로 특정 범위를 지정하고 어떤 단어가 올지 학습 ex) 앞뒤 두칸 or 세칸

In [13]:
#https://huggingface.co/beomi/kcbert-large

## 트랜스포머 살펴보기
트랜스포머는 2017년 구글이 제안한 시퀀스-투-시퀀스 모델입니다. 최근 자연어 처리에서는 BERT나 GPT 같은 트랜스포머 기반 언어모델이 각광받고있습니다.   

시퀀스 투 스퀀스
- 트랜스포머란 기계번역 등 시퀀스-투-시퀀스(단어-단어) 과제를 수행하기위한 모델 
- 시퀀스 투 스퀀스는 특정 속성을 지닌 시퀀스를 다른 속성의 시퀀스로 변환하는 작업
- 기계 번역을 예로 보자

- 소스 시퀀스의 길이와 타깃 시퀀스의 길이가 다를때,
- 시퀀스 투 스퀀스 태스크는 소스와 타깃의 길이가 달라도 과제 수행하는데문제가 없어야함

## 인코더와 디코더
  - 트랜스포머는 시퀀스 투 스퀀스과제 수행에 특화된 모델.
  - 임의의 시퀀스를 해당 시퀀스와 속성이 다른 시퀀스로 변환하나는 작업이라면 꼭 기계번역이 아니라도 수행가능
  - 시퀀스 투 스퀀스과제를 수행하는 모델은 대개 인코더와 디코더 두개로 나눔
  - 인코더는 소스 스퀀스의 정보를 압축해 디코더로 보내주는 역할 -> 압축하는 과정을 인코딩이라고함
  - 디코더는 인코더가 보내준 소스 시퀀스를 받아 타깃 시퀀스를 생성 이과정을 디코딩이라함
  

## 셀프 어텐션
- 셀프 어텐션이 여러개일경우 멀티 헤드 어텐션이라 한다.
- 트랜스포머 경쟁력의 원천은 셀프 어텐션이다.
  - 어텐션은 시퀀스입력에 수행하는 기계학습 방법의 일종
  - 어텐션은 시퀀스 요소들 가운데 태스크 수행에 중요한 요소에 집중하고 그렇지않은 요소는 무시해 수행성능을 높인다.
  - 어텐션은 기계번역과정에 처음 도입

### 합성곱 신경망과 비교
- CNN은 합성곱 필터를 이용해 시퀀스의 지역적인 특징을 잡아내는모델
- 언어는 기본적으로 시퀀스이고 특정 단어 기준 주변 문백이 의미 형성에 중요한 역할을 하므로 CNN 이 자연어 처리에 많이 사용된다.
- CNN은 합성곱 필터 크기를 넘어서는 문맥은 읽어내기 어렵다는 단점이있다.

### 순환 신경망과 비교
- RNN 역시 시퀀스 정보를 압축하는 데 강점이 있는 구조
- 소스 언어 시퀀스인 어제, 카페, 갔었어, 거기, 사람, 많더라를 인코딩해야하한다고 가정, RNN은 그림 3-18과 같이 소스 시퀀스를 차례대로 처리한다.

- RNN은 시퀀스 길이가 길어질 수록 정보 압축에 문제가 발생
  - 장기의존성문제
  - 특정 단어 정보를 과도하게 반영한 전체 정보 왜곡(기계 번역을 할 때 RNN으르 사용한다면 인코더가 디코더로 넘기는 정보는 소스 시퀀스의 마지막인 많더라라는 단어의 의미가 많이 반영될 수 밖에 없다.)

### 어텐션과 비교
 - 다음 그림을 보면 cafe에 대응하는 소스 언어의 단어는 카페이고 이는 소스 시퀀스의 초반부에 등장한 상황.
 - cafe라는 단어를 디코딩해야할때 카페를 반드시 참조해야하는데 어텐션이 없는 단순 RNN을 사용하면 워낙 초반에 입력된 단어라 모델이 잊었을 가능성이 있고, 이 때문에 번역 품질이 낮아짐

### 셀프 어텐션 특징 및 장점
 - 셀프 어텐션은 자기 자신에 수행하는 어텐션이다. 다음 그림을 보면 입력 시퀀스가 어제, 카페, 갔었어, 거기, 사람, 많더라 일때 거기라는 단어가 어떤 의미를 가지는지 계산하는 상황이다.

 - 잘 학습된 셀프 어텐션 모델이라면 거기에 대응하는 장소는 카페라는 사실을 알아챌 수 있을 것이고 뿐만 아니라 거기는 갔었어와도 연관이 있음을 확인할 수 있다.
 - 트랜스포머 인코더 블록 내부에서는 이처럼 거기라는 단어를 인코딩할 때  카페, 갔었어 라는 단어의 의미를 강조해서 반영
 - 그림 3-12은 입력 시퀀스가 어제, 카페, 갔었어, 거기, 사람, 많더라 일때 카페라는 단어가 어떤 의미를 가지는지 계산하는 상황이다. 트랜스포머 인코더 블록은 카페라는 단어를 인코딩할때 거기, 갔었어라는 단어의 의미를 다른 단어들보다 더 강하게 반영

### 셀프 어텐션 계산 예시
- 셀프어텐션은 쿼리, 키, 밸류 세가지 요소가 서로 영향을 주고받음
  - 쿼리 무언가를 물어보는 주체, 단어 x가 
  - 키 : 대응하는 값
  - 밸류 : 실제 정답

- 셀프 어텐션은 쿼리 단어 각각에 대해 모든 키 단어와 얼마나 유기적인 관계를 맺고 있는지 그 합이 1인 확률 값으로 나타낸다. 
- 카페라는 쿼리 단어와 가장 높은 키 단어는 거기라는 점(0,4)을 확인 할 수 있음

### 모델 입력과 출력
- 트랜스포머 모델의 전체 구조에서 인코더 입력만 떼어 낸 그림
- 모델 입력을 만드는 계층을 입력층이라한다.



In [14]:
import torch
x = torch.tensor([
    [1.0, 0.0, 1.0, 0.0],
    [0.0, 2.0, 0.0, 2.0],
    [1.0, 1.0, 1.0, 1.0]
])

w_query = torch.tensor([
    [1.0, 0.0, 1.0],
    [1.0, 0.0, 0.0],
    [0.0, 0.0, 1.0],
    [0.0, 1.0, 1.0]
])

w_key = torch.tensor([
    [0.0, 0.0, 1.0],
    [1.0, 1.0, 0.0],
    [0.0, 1.0, 0.0],
    [1.0, 1.0, 0.0]
])


w_value = torch.tensor([
    [0.0, 2.0, 0.0],
    [0.0, 3.0, 0.0],
    [1.0, 0.0, 3.0],
    [1.0, 1.0, 0.0]
])

In [15]:
# 쿼리, 키, 밸류 만들기
keys = torch.matmul(x, w_key)
querys = torch.matmul(x, w_query)
value = torch.matmul(x, w_value)

In [16]:
print('querys:\n', querys)
print('key:\n', keys)
print('values:\n', value)

querys:
 tensor([[1., 0., 2.],
        [2., 2., 2.],
        [2., 1., 3.]])
key:
 tensor([[0., 1., 1.],
        [4., 4., 0.],
        [2., 3., 1.]])
values:
 tensor([[1., 2., 3.],
        [2., 8., 0.],
        [2., 6., 3.]])


In [17]:
# 어텐션 스코어 만들기
attn_scores = torch.matmul(querys, keys.T) # 전치
print('attn_scores:\n', attn_scores)

attn_scores:
 tensor([[ 2.,  4.,  4.],
        [ 4., 16., 12.],
        [ 4., 12., 10.]])


In [18]:
# 소프트 맥스 확률값 만들기
import numpy as np
from torch.nn.functional import softmax
key_dim_sqrt = np.sqrt(keys.shape[-1])
attn_probs = softmax(attn_scores / key_dim_sqrt, dim=-1)

print('key_dim_sqrt:\n', key_dim_sqrt)
print('attn_prods: \n', attn_probs)

key_dim_sqrt:
 1.7320508075688772
attn_prods: 
 tensor([[1.3613e-01, 4.3194e-01, 4.3194e-01],
        [8.9045e-04, 9.0884e-01, 9.0267e-02],
        [7.4449e-03, 7.5471e-01, 2.3785e-01]])


In [19]:
# 소프트 맥스 확률과 밸류를 가중합하기
weighted_values = torch.matmul(attn_probs, value)
print('weighted_values:\n', weighted_values)

weighted_values:
 tensor([[1.8639, 6.3194, 1.7042],
        [1.9991, 7.8141, 0.2735],
        [1.9926, 7.4796, 0.7359]])


In [20]:
x = torch.tensor([2,1])
w1 = torch.tensor([[3,2,-4],[2,-3,1]])
b1 = 1
w2 = torch.tensor([[-1,1],[1,2],[3,1]])
b2 = -1

In [21]:
# 피드포워드 뉴럴네트워크 계산 예시
h_preact = torch.matmul(x, w1) + b1
h = torch.nn.functional.relu(h_preact)
y = torch.matmul(h,w2) + b2
print(f'h_preact :{h_preact}')
print(f'h: {h}')
print(f'y: {y}')

h_preact :tensor([ 9,  2, -6])
h: tensor([9, 2, 0])
y: tensor([-8, 12])


In [22]:
input = torch.tensor([[1.0,2.0,3.0], [1.0,1.0,1.0]])
# 입력(input)의 모양은 배치 크기(2) x 피처의 차원수(3)가 된다.
m = torch.nn.LayerNorm(input.shape[-1])
# torch.nn.LayerNorm(input.shape[-1])이라는 말은 피처 대상으로 레이어 정규화를 수행
output = m(input)
print(f'output: {output}')

output: tensor([[-1.2247,  0.0000,  1.2247],
        [ 0.0000,  0.0000,  0.0000]], grad_fn=<NativeLayerNormBackward0>)


In [23]:
# 레이어 정규화 예시
print(f'm.weight: {m.weight}') # Gamma에 대응
print(f'm.bias: {m.bias}') # beta에 대응

m.weight: Parameter containing:
tensor([1., 1., 1.], requires_grad=True)
m.bias: Parameter containing:
tensor([0., 0., 0.], requires_grad=True)


In [24]:
# dropout
m = torch.nn.Dropout(p=0.2)
input = torch.randn(1,10)
output = m(input)

In [25]:
print(input)
print(output)

tensor([[ 2.8684,  0.1627, -1.1451,  0.3779,  1.6989, -1.4940,  0.6039,  0.1577,
          2.1870, -0.1332]])
tensor([[ 3.5855,  0.0000, -0.0000,  0.4723,  0.0000, -1.8675,  0.7549,  0.1972,
          2.7337, -0.1666]])


## BERT와 GPT
이번 절에서는 트랜스포머 아케텍처를 기본 뼈대로 하는 BERT와 GPT모델의 공통점과 차이점을 중심으로 살펴봅니다. 마지막으로는 트랜스포머 계열 언어모델 최근 경향도 설명합니다.
### 성능이 좋은 번역모델의 층을 분리해서 BERT와 GPT를 만듬
### BERT
- 마스크 언어모델로 문장 중간에 빈칸을 반들고 해당 빈칸에 어떤 단어가 적절할지 맞추는 과정에서 프리트레인 한다. 빈칸 앞뒤 문맥을 모두 살필 수 있다는 점에서 양방향으로 동작.
- 인코더 층만 사용
- 문장의 의미를 추출하는데 강점

### GPT
- 언어모델로 이전 단어들이 주어졌을 때 다음 단어가 무엇인지 맞추는 과정에서 프리트레인한다. 문장 시작부터 순차적으로 계싼한다는 점에서 일방향으로 동작
- 디코더 층만 사용
- 문장 생성에 강점

## 파인튜닝

- 트랜스포머 계열 언어모델을 사용함
- 프리트레인을 마친 언어 모델 위에 작은 모듈을 조금 더 쌓아 태스크를 수행하는 구조
- 문서 분류, 개체명 인식 등 다운스트림 데이터로 프리트레인 마친 BERT와 그 위의 작은 모듈을 포함한 전체 모델을 업데이트하는 과정을 파인튜닝이라한다.
- BERT의 출력결과 가운데 어떤 걸 사용하느냐에 따라 두 가지 방식으로 나눠짐.

* 프리트레인 : 대규모 말뭉치를 학습 시켜 임베딩을 미리 만들어 놓은것.

문장 벡터 확용 : 문서 분류 등
- 문서 분류를 수행하는 모델을 만든다고 하면 그림과 같은 모양이 된다.

In [26]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 8.2 MB/s 
     |████████████████████████████████| 6.6 MB 46.0 MB/s 
     |████████████████████████████████| 120 kB 65.7 MB/s 


## 토크나이저 초기화
- BERT 모델의 입력값을 만들려면 토크나이저부터 선언
- 이준범 님이 허깅페이스에 등록한 kcbert-base 모델이 쓰는 토크나이저를 선언

In [27]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    "beomi/kcbert-base",
    do_lower_case=False,
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

## 모델 초기화
- 사용 대상 BERT 모델이 프리트레인 할 때 썼던 토크나이저를 그대로 사용해야 벡터 변환에 문제가 없다.
- 모델과 토크나이저의 토큰화 방식이 다를 경우 모델이 엉뚱한 결과를 출력한다.
- 모델을 선언할 때 토크나이저 초기화와 동일한 모델 이름으로 적용

In [28]:
from transformers import BertConfig, BertModel
pretrained_model_config = BertConfig.from_pretrained(
    'beomi/kcbert-base'
)
model = BertModel.from_pretrained(
    'beomi/kcbert-base',
    config=pretrained_model_config,
)

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [29]:
pretrained_model_config

BertConfig {
  "_name_or_path": "beomi/kcbert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 300,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.22.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30000
}

In [30]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30000, 768, padding_idx=0)
    (position_embeddings): Embedding(300, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

## 입력값 만들기
- 두개의 입력 문장 각각에 대해 워드피스 토큰화를 수행한 뒤 이를 토큰 인덱스로 변환한 결고가가 input_ids
- BERT ㅗㅁ델은 문장 시작에 CLS, 끝에 SEP 라는 스페셜 토큰을 추가하기 때문에 문장 두개 모두 앞뒤에 이들 토큰에 대응하는 인덱스 2,3이 붙여져있음
- max_length=10 : 토큰 최대길이를 10으로 설정
- padding='max_length : max_length보다 토큰길이가 이보다 짧으면 최대길이에 맞게 패딩
- truncation=True : max_length보다 토큰길이가 이보다 크면 뒤에 단어 제거

In [33]:
sentences = ['안녕 하세요', '하이 !']
features = tokenizer(
    sentences,
    max_length =10,
    padding = 'max_length',
    truncation=True
)

- input_ids의 길이는 두 문장 모두 10
- attention_mask는 패딩이 아닌 토큰이1 패딩인 토큰이 0으로 실제 토큰이 자리하는지 아닌지 정보를 나타낸다.
- token_type_ids는 세그먼트 정보로 파인튜닝을 실시할때는 모두 0

In [34]:
features # 2와 3이 시작 종료 토큰

{'input_ids': [[2, 19017, 9287, 3, 0, 0, 0, 0, 0, 0], [2, 15830, 5, 3, 0, 0, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 0, 0, 0, 0, 0, 0]]}

### BERT에 적용
- 딥러닝 프레임워크로 파이토치를 사용
- 파이토치 모델의 입력값 자료형은 파이토치에서 제공하는 텐서여야한다.
- 파이썬 리스트 형태의 features를 텐서로 변환

In [35]:
# 피쳐들 토치 텐서로 변환
features = {k: torch.tensor(v) for k, v in features.items()}
features

{'input_ids': tensor([[    2, 19017,  9287,     3,     0,     0,     0,     0,     0,     0],
         [    2, 15830,     5,     3,     0,     0,     0,     0,     0,     0]]),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 0, 0, 0, 0, 0, 0]])}

In [36]:
outputs = model(**features)

- 단어 수준 임베딩 결과
- 단어 수준 임베딩 결과는 개체명 인식 과제 같이 단어별로 수행해야 하는 태스크에 활용

In [37]:
outputs.last_hidden_state

tensor([[[-0.2074, -0.6982,  1.7594,  ..., -0.2775,  0.8582,  1.2953],
         [-0.9689, -0.6768,  1.0893,  ..., -0.6624, -0.4993,  0.7456],
         [-0.9515, -0.0449,  0.7296,  ...,  0.2030, -0.3377,  2.1919],
         ...,
         [-0.8945, -0.3541,  1.9087,  ..., -0.1381, -0.0831, -0.1006],
         [-0.7601, -0.1213,  1.5564,  ...,  1.1778,  0.2080,  0.3530],
         [-0.8688, -0.3187,  1.6059,  ..., -0.0544,  0.1962,  0.1550]],

        [[ 0.9371, -1.4749,  1.7351,  ..., -0.3426,  0.8050,  0.4031],
         [ 1.6095, -1.7269,  2.7936,  ...,  0.3100, -0.4787, -1.2491],
         [ 0.4861, -0.4569,  0.5712,  ..., -0.1769,  1.1253, -0.2756],
         ...,
         [ 1.2362, -0.6181,  2.0906,  ...,  1.3677,  0.8132, -0.2742],
         [ 0.5409, -0.9652,  1.6237,  ...,  1.2395,  0.9185,  0.1782],
         [ 1.9001, -0.5859,  3.0156,  ...,  1.4967,  0.1924, -0.4448]]],
       grad_fn=<NativeLayerNormBackward0>)

- torch.Size([2,10,768]): 문장 두개에 속한 각각의 토큰(최대길이10)을 768차원짜리의 벡터로 변환했다는 의미

In [38]:
outputs.last_hidden_state.shape

torch.Size([2, 10, 768])

In [39]:
outputs.pooler_output

tensor([[-0.4127, -0.1003,  0.0930,  ...,  0.1367,  0.2863, -0.9833],
        [-0.9221,  0.2969, -0.0110,  ...,  0.4291,  0.0311, -0.9955]],
       grad_fn=<TanhBackward0>)

In [41]:
outputs.pooler_output.shape

torch.Size([2, 768])